# Cálculo de Energia Livre em Catálise usando VASP e ASE

Autor: [Prof. Elvis do A. Soares](https://github.com/elvissoares) <br>
Adaptado: [PhD. Hugo de Lacerda C. Neto](https://github.com/hugo-neto)

Contato: [elvis@peq.coppe.ufrj.br](mailto:elvis@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/) <br>
Contato: [hneto@peq.coppe.ufrj.br](mailto:hneto@peq.coppe.ufrj.br) - [Programa de Engenharia Química, PEQ/COPPE, UFRJ, Brasil](https://www.peq.coppe.ufrj.br/)

---

In [1]:
import os
# Definindo o path para os arquivos de potencial de pseudopotenciais do VASP
# Certifique-se de que o caminho esteja correto para o seu sistema
os.environ['VASP_PP_PATH'] = '/home/hugo/Documents/Programs/VASP/vasp-6.5.1/pp'
os.environ['ASE_VASP_COMMAND'] = 'mpirun -np 1 vasp_std_gpu'
os.environ['NO_STOP_MESSAGE'] = '1' # to avoid warning from mpirun

# Importando o VASP calculator do ASE
from ase.calculators.vasp import Vasp

from ase import Atoms
from ase.io import write, read
from ase.visualize import view

import numpy as np

# Water Gas Shift Reaction

$H_2 O + CO \to CO_2 + H_2$ 

Valores experimentais

$\Delta G_r = -28.6\ \text{kJ/mol}$

$\Delta H_r = -41.2\ \text{kJ/mol}$

In [2]:
h2o = Atoms('H2O',
            positions=[[-0.768, 0.000, 0.595],
                       [0.768, 0.000, 0.595],
                       [0.000, 0.000, 0.000]]) # geometria otimizada com PBE
h2o.center(vacuum=6.0)  
h2o.pbc = True

calc_relax = Vasp(
    directory='h2o_relax',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-6, ediffg=-0.01,
    isif=2, ibrion=2, nsw=100,
    nelm=100,
    lwave=True, lcharg=True, lvtot=True,
    atoms=h2o
)
calc_relax.calculate(h2o) # demora por volta de 10 s

E_h2o = h2o.get_potential_energy()

print(f'Optimized energy of H2O: {E_h2o:.3f} eV')

Optimized energy of H2O: -14.219 eV


In [3]:
calc_vib = Vasp(
    directory='h2o_vib',
    xc='PBE',
    encut=450,
    ismear=0, sigma=0.05,
    ediff=1e-8,
    ibrion=6,           # finite differences with symmetry
    lreal=False,
    lwave=False, lcharg=False,
    atoms=h2o
)

calc_vib.calculate(h2o)

vib_water = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = vib_water.get_frequencies()

print(f'Vibrational frequencies of H2O (cm⁻¹):')
vib_energies_h2o = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energies_h2o.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

Vibrational frequencies of H2O (cm⁻¹):
f_v: 1584.717 cm⁻¹, E_v: 0.196 eV
f_v: 3728.225 cm⁻¹, E_v: 0.462 eV
f_v: 3840.285 cm⁻¹, E_v: 0.476 eV


In [4]:
from ase.thermochemistry import IdealGasThermo

P = 101325. # Pa
T = 298.15  # K

h2o_thermo = IdealGasThermo(vib_energies=vib_energies_h2o,
                        potentialenergy=E_h2o, atoms=h2o,
                        geometry='nonlinear', symmetrynumber=2, spin=0)

G_h2o = h2o_thermo.get_gibbs_energy(temperature=T, pressure=P, verbose=False)
H_h2o = h2o_thermo.get_enthalpy(temperature=T, verbose=False)

# Reação de WGS com Catalisador de Pt(111)

Reações de Adsorção

$ H_2 O + ^* \rightleftarrows H_2 O^*$

$ H_2 O(ads) \rightleftarrows HO(ads) + H(ads)$

**Ref:** 
- Grabow, L. C., Gokhale, A. A., Evans, S. T., Dumesic, J. A., & Mavrikakis, M. (2008). _Mechanism of the water gas shift reaction on Pt: First principles, experiments, and microkinetic modeling._ The Journal of Physical Chemistry C, 112(12), 4608-4617. https://pubs.acs.org/doi/pdf/10.1021/jp7099702
- Gokhale, A. A., Dumesic, J. A., & Mavrikakis, M. (2008). _On the mechanism of low-temperature water gas shift reaction on copper._ Journal of the American Chemical Society, 130(4), 1402-1414. https://pubs.acs.org/doi/pdf/10.1021/ja0768237

## Equilibrando Geometria do Catalisador

Cristal bulk de Pt

In [5]:
from ase.lattice.cubic import FaceCenteredCubic

# bulk system
Pt_crystal = FaceCenteredCubic(size=(1, 1, 1), 
                               symbol='Pt', 
                               latticeconstant=3.967, 
                               pbc=True)

calc_relax = Vasp(directory ='Pt_relax',
                xc          ='PBE',
                kpts        =[6, 6, 6],  # specifies k-points
                encut       =450,
                ismear      =0, 
                sigma       =0.05,
                ediff =1e-6, ediffg=-0.01,
                isif=7, ibrion=2, nsw=100, # ionic relaxation (ISIF=7: relax cell volume only)
                atoms=Pt_crystal
                )
calc_relax.calculate(Pt_crystal) # demora por volta de 2 s

E_Pt  = Pt_crystal.get_potential_energy()
a0_Pt = Pt_crystal.cell.cellpar()[0]

print(f'Optimized energy of Pt: {E_Pt:.3f} eV')
print(f"Lattice length: {a0_Pt:.3f} Å")

Optimized energy of Pt: -24.458 eV
Lattice length: 3.967 Å


Superfície Pt111

In [ ]:
from ase.build import fcc111

# Superfície Pt111
Pt111_slab = fcc111("Pt", 
                    size=(2,2,3), 
                    a=a0_Pt)
Pt111_slab.center(vacuum=5.0, 
                  axis=2)
Pt111_slab.pbc = True

calc_relax = Vasp(directory ='Pt111_slab_relax',
                xc          ='PBE',
                kpts        =[6, 6, 1], # specifies k-points
                encut       =450,
                ismear=0, sigma=0.05,
                ediff=1e-6, ediffg=-0.01,
                ibrion=-1,              # just SCF (no ionic relaxation)
                atoms=Pt111_slab
                )
calc_relax.calculate(Pt111_slab) # demora por volta de 1 min (RTX 5060)

E_Pt111 = Pt111_slab.get_potential_energy()

print(f'Optimized energy of Pt111: {E_Pt111:.3f} eV')

Optimized energy of Pt111: -67.924 eV


In [7]:
from ase.visualize import view

view(Pt111_slab, viewer='x3d')

## Reação 1: Water Adsorption

$ H_2 O + * \to H_2 O*$ 

Calculando equilíbrio da geometria da molécula $H_2 O$ adsorvida na superfície Pt111

In [8]:
from ase.constraints import FixAtoms
from ase.build import add_adsorbate
# Adsorvendo a molécula de H2O na superfície Pt111

H2Oads_slab = fcc111("Pt", 
                     size=(2,2,3), 
                     a=a0_Pt)
H2Oads_slab.center(vacuum=5.0, 
                   axis=2)
H2Oads_slab.pbc = True

h2o = Atoms('H2O',
            positions=[[-0.768, 0.595, 0.000],
                       [0.768, 0.595, 0.000],
                       [0.000, 0.000, 0.000]]) # geometria otimizada com PBE

h2o.rotate('z', 104.46/2) # rotaciona a molécula em 60 graus em torno do eixo z

add_adsorbate(H2Oads_slab,  
              adsorbate=h2o, 
              height=2.45, 
              position = (4.50, 2.18), 
              mol_index=2 )

# fixando os átomos de Pt
constraint = FixAtoms(mask=[atom.symbol=='Pt' for atom in H2Oads_slab])
H2Oads_slab.set_constraint([constraint])

calc_relax = Vasp(directory ='Pt111_H2O_relax',
                xc          ='PBE',
                kpts        =[6, 6, 1],  # specifies k-points
                encut       =450,
                ismear=0, sigma=0.05,
                ediff=1e-6, ediffg=-0.01,
                isif=0, ibrion=2, nsw=100, # ionic relaxation (ISIF=2: relax atoms position only)
                atoms=H2Oads_slab
                )
calc_relax.calculate(H2Oads_slab) # demora por volta de 210 min (RTX 5060)

E_Pt111_H2O = H2Oads_slab.get_potential_energy()

print(f'Optimized energy of H2O in top position of Pt111: {E_Pt111_H2O:.3f} eV')


Optimized energy of H2O in top position of Pt111: -82.353 eV


In [9]:
E_ads_Pt111_H2O = E_Pt111_H2O - E_Pt111 - E_h2o

print(f'Adsorption energy of H2O on Pt111: {E_ads_Pt111_H2O:.3f} eV')

Adsorption energy of H2O on Pt111: -0.210 eV


In [10]:
view(H2Oads_slab, viewer='x3d')

Cálculando o espéctro da molécula de H2O

In [ ]:
calc_vib = Vasp(
    directory   ='Pt111_H2O_vib',
    xc          ='PBE',
    kpts        =[6, 6, 1],
    encut       =450,
    ismear=0, sigma=0.05,
    ediff=1e-8,
    ibrion=5,nfree=2, nsw=1,           # finite differences with selective dynamics (Pt atoms fixed)
    lreal=False,lwave=False, lcharg=False,
    atoms=H2Oads_slab
)

calc_vib.calculate(H2Oads_slab) # Aprox. 80 min (RTX 5060)

vib_Pt111_h2o = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = vib_Pt111_h2o.get_frequencies()

print(f'Vibrational frequencies of H2O on Pt111 (cm⁻¹):')

vib_energie_Pt111_h2o = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energie_Pt111_h2o.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

Vibrational frequencies of H2O on Pt111 (cm⁻¹):
f_v: 496.157 cm⁻¹, E_v: 0.062 eV
f_v: 550.805 cm⁻¹, E_v: 0.068 eV
f_v: 1546.702 cm⁻¹, E_v: 0.192 eV
f_v: 3576.449 cm⁻¹, E_v: 0.443 eV
f_v: 3674.422 cm⁻¹, E_v: 0.456 eV


In [12]:
h2o_Pt111_thermo = IdealGasThermo(vib_energies=vib_energie_Pt111_h2o,
                                  potentialenergy=E_Pt111_H2O, 
                                  atoms=H2Oads_slab,
                                  geometry='nonlinear', 
                                  symmetrynumber=2, 
                                  spin=0)

G_h2o_Pt111 = h2o_Pt111_thermo.get_gibbs_energy(temperature=T, 
                                                pressure=P, 
                                                verbose=False)
H_h2o_Pt111 = h2o_Pt111_thermo.get_enthalpy(temperature=T, 
                                            verbose=False)

In [13]:
Hads_h2o = H_h2o_Pt111 - H_h2o - E_Pt111
      
print(f'ΔH h2o adsorption = {Hads_h2o:.3f} eV')

ΔH h2o adsorption = -0.156 eV


## Reação 2: Water Dessociation

$ H_2 O(ads) \to OH(ads) + H(ads)$ 

Criando H e HO adsorvidos

In [14]:
HO_H_ads_slab = fcc111("Pt", 
                     size=(2,2,3), 
                     a=a0_Pt)
HO_H_ads_slab.center(vacuum=5.0, 
                   axis=2)
HO_H_ads_slab.pbc = True

ho  = Atoms('HO',
            positions=[[0.000, 0.999, 0.000],
                       [0.000, 0.000, 0.000]]) # geometria https://pubs.acs.org/doi/pdf/10.1021/jp7099702

ho.rotate(90,'x') # rotaciona a molécula em 90 graus

add_adsorbate(HO_H_ads_slab,  
              adsorbate=ho, 
              height=2.45, 
              position = (3.00, 2.18), 
              mol_index=1  # Grupo de Simetria C∞v
              )

h = Atoms('H',
          positions=[[0.000, 0.000, 0.000]])

add_adsorbate(HO_H_ads_slab,  
              adsorbate=h, 
              height=1.1, 
              position = (1.6, 1.20), 
              mol_index=0  # Grupo de Simetria Oh (simetria)
              ) # OBS: Coloquei "0" e foi ...

# fixando os átomos de Pt - assume que o sólido não se meche para reduzir custo computacional
constraint = FixAtoms(mask=[atom.symbol=='Pt' for atom in HO_H_ads_slab])
HO_H_ads_slab.set_constraint([constraint])

calc_relax = Vasp(directory ='Pt111_HO_H_relax',
                xc          ='PBE',
                kpts        =[6, 6, 1],  # specifies k-points
                encut       =450,
                ismear=0, sigma=0.05,
                ediff=1e-6, ediffg=-0.01,
                isif=0, ibrion=2, nsw=100, # ionic relaxation (ISIF=2: relax atoms position only)
                atoms=HO_H_ads_slab
                )

Escolhi como "chute inicial" das posições das espécies dissociadas <br>
como estando próximo da posição ótima de 10.1021/jp7099702

In [15]:
view(HO_H_ads_slab, viewer='x3d')

In [ ]:
calc_relax.calculate(HO_H_ads_slab) # Aprox. 240 min (RTX 5060)

E_Pt111_HO_H = HO_H_ads_slab.get_potential_energy()

print(f'Optimized energy of HO + H in top position of Pt111: {E_Pt111_H2O:.3f} eV')

Optimized energy of HO + H in top position of Pt111: -82.353 eV


In [17]:
E_des_Pt111_HO_H = E_Pt111_HO_H - E_Pt111_H2O 

print(f'Dessociation energy of H2O on Pt111: {E_des_Pt111_HO_H:.3f} eV')

Dessociation energy of H2O on Pt111: 0.613 eV


Angle (106.2°) e distance (2.02) próxima da REF: 10.1021/jp7099702 <br>
Abrir em VESTA para mais detalhes <br><br>
Esse valor pode ser um mínimo local ou global. <br>
Não fiz uma varredura das possibilidades ...

In [18]:
view(HO_H_ads_slab, viewer='x3d')

Calculando os Espéctros

In [ ]:
calc_vib = Vasp(
    directory   ='Pt111_HO_H_vib',
    xc          ='PBE',
    kpts        =[6, 6, 1],
    encut       =450,
    ismear=0, sigma=0.05,
    ediff=1e-8,
    ibrion=5,nfree=2, nsw=1,           # finite differences with selective dynamics (Pt atoms fixed)
    lreal=False,lwave=False, lcharg=False,
    atoms=HO_H_ads_slab
)

calc_vib.calculate(HO_H_ads_slab) # Aprox. 80 min (RTX 5060)

vib_Pt111_ho_h = calc_vib.get_vibrations()

# Frequencies (cm⁻¹)
freqs = vib_Pt111_ho_h.get_frequencies()

print(f'Vibrational frequencies of HO and H on Pt111 (cm⁻¹):')

vib_energie_Pt111_ho_h = []
for freq in freqs:
    if freq.real > 200:
        vib_energy = (freq.real * 1.23981e-4)  # Convert cm⁻¹ to eV
        vib_energie_Pt111_ho_h.append(vib_energy)
        print(f'f_v: {freq.real:.3f} cm⁻¹, E_v: {vib_energy:.3f} eV')

Vibrational frequencies of HO and H on Pt111 (cm⁻¹):
f_v: 473.921 cm⁻¹, E_v: 0.059 eV
f_v: 512.227 cm⁻¹, E_v: 0.064 eV
f_v: 630.825 cm⁻¹, E_v: 0.078 eV
f_v: 928.596 cm⁻¹, E_v: 0.115 eV
f_v: 1127.541 cm⁻¹, E_v: 0.140 eV
f_v: 3627.468 cm⁻¹, E_v: 0.450 eV


In [20]:
ho_h_Pt111_thermo = IdealGasThermo(vib_energies=vib_energie_Pt111_ho_h,
                                  potentialenergy=E_Pt111_HO_H, 
                                  atoms=HO_H_ads_slab,
                                  geometry='nonlinear', # Simetria não linear
                                  symmetrynumber=1,     # C1 - sem simetria global
                                  spin=0)               # Assume-se conjunto sem é desemparelhado

G_ho_h_Pt111 = ho_h_Pt111_thermo.get_gibbs_energy(temperature=T, 
                                                pressure=P, 
                                                verbose=False)
H_ho_h_Pt111 = ho_h_Pt111_thermo.get_enthalpy(temperature=T, 
                                            verbose=False)

In [21]:
H_dis_h2o = H_ho_h_Pt111 - H_h2o_Pt111
print(f'ΔH h2o dissociation = {H_dis_h2o:.3f} eV')

G_dis_h2o = G_ho_h_Pt111 - G_h2o_Pt111
print(f'ΔG h2o dissociation = {G_dis_h2o:.3f} eV')

ΔH h2o dissociation = 0.462 eV
ΔG h2o dissociation = 0.435 eV


OBS: Difere muito de 10.1021/jp7099702 (table 3) <br>
Possívelmente pq o dado é experimental e em Temperatura diferente

## Resultado Final


| Reação                                         | $\Delta E$ (eV)  | $\Delta H$ (eV) |
|------------------------------------------------|----------|------------|
| $ H_2 O + ^* \rightleftarrows H_2 O^*$         |  -0.210  | -0.156     |
| $ H_2 O(ads)\rightleftarrows OH(ads) + H(ads)$ |  +0.612  | +0.462     |
